In [1]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

#from colorama import Fore, Back, Style

from IPython.display import clear_output


#warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
# print(len(backend.tensorflow_backend._get_available_gpus()) > 0)

#warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

sys.path.insert(1, '..//components//')
import load_feat_directories

C:\Users\yg9ca\AppData\Local\Continuum\anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Using TensorFlow backend.


In [2]:
all_noised_npy = load_feat_directories.allnoised_npy
all_noised_npy_test = load_feat_directories.allnoised_npy_test
home_noised_npy = load_feat_directories.homenoised_npy
home_noised_npy_test = load_feat_directories.homenoised_npy_test

for index in range(0, 5):
    if not os.path.exists(all_noised_npy[index]):
        print(all_noised_npy[index] + ' does not exist. Breaking the loop... ')
        break

    if not os.path.exists(home_noised_npy[index]):
        print(home_noised_npy[index] + 'does not exist. Breaking the loop... ')
        break

In [3]:
def comprise_vector(path):
    vec_to_return = []
    for fname in os.listdir(path):
        current_vec = np.load(path + fname)
        vec_to_return.append(current_vec)

    vec_to_return = np.array(vec_to_return)
    return vec_to_return

def comprise_label(feature_vector, label):
    label_vec_to_ret = []
    length = len(list(feature_vector))
    for index in range(0, length):
        current_label = [label]
        label_vec_to_ret.append(current_label)
    label_vec_to_ret = np.array(label_vec_to_ret)

    return label_vec_to_ret

def float_compatible(input_np):
    x = np.where(input_np >= np.finfo(np.float32).max)
    for index in range(0, len(x[0])):
        x_position = x[0][index]
        y_position = x[1][index]
        input_np[x_position, y_position] = 0.0
    input_np = np.nan_to_num(input_np)

    return input_np

In [4]:
for index in [0, 1, 2, 3, 4]:
    if not os.path.exists(home_noised_npy[index]):
        print(home_noised_npy[index] + 'does not exist.')
    else:
        path = home_noised_npy[index]
        print(path)
        if index == 0:
            h_feature_vector_home = comprise_vector(path)
            h_label_vector_home = comprise_label(h_feature_vector_home, index)
        elif index == 1:
            a_feature_vector_home = comprise_vector(path)
            a_label_vector_home = comprise_label(a_feature_vector_home, index)
        elif index == 2:
            n_feature_vector_home = comprise_vector(path)
            n_label_vector_home = comprise_label(n_feature_vector_home, index)
        elif index == 3:
            s_feature_vector_home = comprise_vector(path)
            s_label_vector_home = comprise_label(s_feature_vector_home, index)
        else:
            o_feature_vector_home = comprise_vector(path)
            o_label_vector_home = comprise_label(o_feature_vector_home, index)

for index in [0, 1, 2, 3, 4]:
    
    if not os.path.exists(home_noised_npy_test[index]):
        print(home_noised_npy_test[index] + 'does not exist.')
    else:
        path = home_noised_npy_test[index]
        print(path)
        if index == 0:
            h_feature_vector_home_test = comprise_vector(path)
            h_label_vector_home_test = comprise_label(h_feature_vector_home_test, index)
        elif index == 1:
            a_feature_vector_home_test = comprise_vector(path)
            a_label_vector_home_test = comprise_label(a_feature_vector_home_test, index)
        elif index == 2:
            n_feature_vector_home_test = comprise_vector(path)
            n_label_vector_home_test = comprise_label(n_feature_vector_home_test, index)
        elif index == 3:
            s_feature_vector_home_test = comprise_vector(path)
            s_label_vector_home_test = comprise_label(s_feature_vector_home_test, index)
        else:
            o_feature_vector_home_test = comprise_vector(path)
            o_label_vector_home_test = comprise_label(o_feature_vector_home_test, index)


C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Happy_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Angry_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Neutral_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Sad_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Other_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Happy_npy_test//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Angry_npy_test//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Neutral_npy_test//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Sad_npy_test//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberate

In [5]:
# Load training npy files
featureSet = float_compatible(np.vstack((h_feature_vector_home, a_feature_vector_home, n_feature_vector_home, s_feature_vector_home, o_feature_vector_home)))
Label = (np.vstack((h_label_vector_home, a_label_vector_home, n_label_vector_home, s_label_vector_home, o_label_vector_home)))

Label[Label == 0] = 0
Label[Label == 1] = 1
Label[Label == 2] = 2
Label[Label == 3] = 3
Label[Label == 4] = 4

Label = to_categorical(Label)
#featureSet = np.split(featureSet, np.array([NumofFeaturetoUse]), axis = 2)[0]
print('training data: ' + str(featureSet.shape))
print('training label: ' + str(Label.shape))

# Load testing npy files
featureSet_val = float_compatible(np.vstack((h_feature_vector_home_test, a_feature_vector_home_test, n_feature_vector_home_test, s_feature_vector_home_test, o_feature_vector_home_test)))
Label_val = (np.vstack((h_label_vector_home_test, a_label_vector_home_test, n_label_vector_home_test, s_label_vector_home_test, o_label_vector_home_test)))

Label_val[Label_val == 0] = 0
Label_val[Label_val == 1] = 1
Label_val[Label_val == 2] = 2
Label_val[Label_val == 3] = 3
Label_val[Label_val == 4] = 4

Label_val = to_categorical(Label_val)
#featureSet_val = np.split(featureSet_val, np.array([NumofFeaturetoUse]), axis = 2)[0]
print('evaluation data: ' + str(featureSet_val.shape))
print('evaluation label: ' + str(Label_val.shape))

training data: (39916, 48, 272)
training label: (39916, 5)
evaluation data: (8843, 48, 272)
evaluation label: (8843, 5)


In [6]:
from sklearn.manifold import TSNE

In [8]:
X_embedded = TSNE(n_components=1).fit_transform(h_feature_vector_home[0])

In [12]:
X_embedded

array([[-1069.6829 ],
       [ 1317.7118 ],
       [ 2821.1926 ],
       [-1933.369  ],
       [ -333.25528],
       [-1213.8995 ],
       [ 1496.5409 ],
       [-2995.4712 ],
       [-2676.242  ],
       [-3774.9136 ],
       [ 3210.4885 ],
       [ -482.16693],
       [ 3552.7417 ],
       [  772.3555 ],
       [ -777.1905 ],
       [-2219.7766 ],
       [-1513.6342 ],
       [  276.29898],
       [ 8414.485  ],
       [ 6514.9272 ],
       [ 5036.372  ],
       [  601.483  ],
       [-4071.3677 ],
       [-2364.6245 ],
       [-2078.213  ],
       [ 2306.5513 ],
       [ 1134.7405 ],
       [-1790.6211 ],
       [  -32.7557 ],
       [-2518.49   ],
       [ 1886.0121 ],
       [-1655.7955 ],
       [-3168.249  ],
       [-1364.0596 ],
       [ -183.35278],
       [  436.1899 ],
       [ -923.0584 ],
       [-2834.299  ],
       [-7328.9795 ],
       [ 2090.159  ],
       [-3353.7478 ],
       [  949.97064],
       [ 2549.7693 ],
       [ 4000.634  ],
       [-3551.1753 ],
       [ -

In [10]:
X_embedded = TSNE(n_components=1).fit_transform(a_feature_vector_home[0])
X_embedded

array([[-1069.6829 ],
       [ 1317.7118 ],
       [ 2821.1926 ],
       [-1933.369  ],
       [ -333.25528],
       [-1213.8995 ],
       [ 1496.5409 ],
       [-2995.4712 ],
       [-2676.242  ],
       [-3774.9136 ],
       [ 3210.4885 ],
       [ -482.16693],
       [ 3552.7417 ],
       [  772.3555 ],
       [ -777.1905 ],
       [-2219.7766 ],
       [-1513.6342 ],
       [  276.29898],
       [ 8414.485  ],
       [ 6514.9272 ],
       [ 5036.372  ],
       [  601.483  ],
       [-4071.3677 ],
       [-2364.6245 ],
       [-2078.213  ],
       [ 2306.5513 ],
       [ 1134.7405 ],
       [-1790.6211 ],
       [  -32.7557 ],
       [-2518.49   ],
       [ 1886.0121 ],
       [-1655.7955 ],
       [-3168.249  ],
       [-1364.0596 ],
       [ -183.35278],
       [  436.1899 ],
       [ -923.0584 ],
       [-2834.299  ],
       [-7328.9795 ],
       [ 2090.159  ],
       [-3353.7478 ],
       [  949.97064],
       [ 2549.7693 ],
       [ 4000.634  ],
       [-3551.1753 ],
       [ -